Predicting the Catgories of the Products ..

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
mapping_data = pd.read_csv("/content/drive/MyDrive/QUANTUM ANALYTICA- CANNABIS/product_category_mappings_new.csv")

In [6]:
#removing special characters and digits form the product name column.
mapping_data['product_name'] = mapping_data['product_name'].str.replace(r'\d+','')

In [8]:
#retaining only english keywords in product names
for i, row in mapping_data.iterrows():
    text_t = ' '.join([w for w in row['product_name'].split() if wordnet.synsets(w)])
    mapping_data.at[i,'product_name'] = text_t

In [11]:
#converting the product_names to lower and remving stopwords if any
for i, row in mapping_data.iterrows():
    word_list_t = row['product_name'].lower().split() 
    filtered_words = [word for word in word_list_t if word not in stopwords.words('english')]
    text_t = ' '.join(filtered_words)
    mapping_data.at[i,'product_name']= text_t

In [13]:
#Tokenization
mapping_data['product_names_token'] = mapping_data['product_name'].apply(lambda x: x.split())

In [15]:
#vectorization
def token_check(x,model):
  """
  1.Check if the token exists in the word2vec model vocab. 
  2.Check if the length of the token is greater than 3 
  """
  token_list=[]
  count_dict= {}
  for i in x:
      if i not in count_dict and i in model.vocab:
        count_dict[i] = 1
      #if len(i) > 3 and i in model.vocab:
        token_list.append(i)
      else:
          continue
  return token_list

def word_vector(tokens,size,model):

  """Averaging the word vectors"""
  
  vec = np.zeros(size).reshape((1, size))
  count = 0
  for word in tokens:
    vec += model[word].reshape((1, size))
    count += 1.
  if count != 0:
      vec /= count
  return vec


#loading the Google Word Embeddings
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

#Load the word2vec pre trained Model to get the word embeddings for each token
model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz',binary=True,limit=1000000)

mapping_data['product_names_token'] = mapping_data['product_names_token'].apply(lambda x:token_check(x,model))

--2021-05-29 16:31:27--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.0.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.0.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  87.4MB/s    in 19s     

2021-05-29 16:31:46 (82.9 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [18]:
mapping_data['length_of_tokens'] = mapping_data['product_names_token'].apply(lambda x: len(x))

In [19]:
new_data = mapping_data[mapping_data['length_of_tokens'] > 0]

In [21]:

new_data.reset_index(drop=True, inplace=True)

wordvec_arrays = np.zeros((len(new_data.product_names_token),300))

for i in range(len(new_data.product_names_token)):
  wordvec_arrays[i,:] = word_vector(new_data.product_names_token[i],300,model)

vectorized_df = pd.DataFrame(wordvec_arrays)

In [22]:
vectorized_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.017822,-0.007568,0.176758,0.070618,0.094482,-0.149536,0.114120,-0.095215,0.040161,0.036133,0.039429,-0.058350,-0.059143,-0.153625,-0.291016,0.138184,0.034180,0.252930,-0.038086,-0.077393,-0.161011,0.331055,0.026123,-0.283691,-0.084656,0.210693,-0.012939,-0.013184,0.138672,-0.073578,0.098633,0.104492,0.182617,0.079895,0.029358,-0.016327,0.147949,0.056549,-0.083130,0.109741,...,0.035034,-0.075562,-0.082031,0.169159,0.114624,0.186279,-0.066406,-0.077759,-0.099609,-0.099365,0.070526,-0.051636,0.082520,0.052612,-0.044434,-0.026855,-0.162598,-0.119751,-0.069092,0.226562,0.111572,-0.003174,0.059570,0.025146,-0.015137,0.099487,-0.041016,-0.015869,-0.086792,-0.053467,-0.067017,-0.016846,-0.158447,-0.073517,0.017090,-0.124512,0.053955,-0.021606,0.035706,-0.252441
1,-0.112305,-0.285645,0.063354,0.031982,-0.238281,0.169922,0.281250,-0.088867,0.307617,-0.044922,-0.100830,-0.033081,0.162109,0.269043,-0.216309,0.286133,-0.035645,0.081299,0.119629,-0.287842,0.264648,0.139343,0.046265,0.098816,-0.123047,-0.051544,-0.041199,0.192871,0.075684,-0.124023,-0.020386,-0.047852,-0.126709,0.207520,0.224609,0.194824,-0.102661,0.237305,-0.063110,-0.205322,...,0.086914,-0.107178,0.125488,0.188965,0.057617,0.317871,-0.134521,-0.082031,-0.227051,-0.090942,0.184570,-0.003540,0.191406,0.166748,-0.053772,0.118042,0.010742,-0.139648,0.208252,0.165771,0.096924,-0.220215,-0.032227,0.237305,-0.159180,-0.040283,-0.146240,0.145508,0.155029,-0.057251,0.049805,-0.055904,0.132812,0.190308,0.030273,0.049316,-0.091675,0.006836,-0.165039,0.044312
2,-0.018311,0.018433,0.126526,0.141357,-0.094727,-0.134766,0.079941,-0.159912,-0.057007,0.104492,-0.043213,0.020874,-0.068176,-0.031067,-0.224854,0.055664,-0.195312,0.185547,-0.151123,-0.202637,-0.013123,0.102661,0.054932,-0.284180,-0.148926,0.012085,-0.139893,0.176025,0.050415,-0.010498,0.019531,0.168213,0.170410,0.022278,-0.020508,-0.029541,0.199219,0.045715,-0.084351,0.051880,...,0.081543,0.002197,-0.206787,-0.004974,0.137451,0.007812,-0.152100,-0.096313,0.154297,-0.063461,0.035645,0.040283,-0.043213,0.069092,0.099609,0.035095,-0.132324,-0.188721,-0.055786,0.143059,-0.071533,0.154785,0.249512,0.010925,0.059021,-0.029053,0.069481,0.007568,-0.202637,-0.039062,-0.009399,-0.076843,-0.135986,0.028412,0.081375,-0.184570,0.155762,-0.002441,0.082092,-0.076973
3,-0.007324,0.156738,0.156738,-0.111694,-0.036316,-0.186401,0.054688,-0.072754,0.213867,0.057861,0.049774,-0.065674,-0.008789,0.024963,-0.257324,-0.178223,-0.027100,0.165039,-0.033123,-0.118835,0.080566,0.055298,0.030396,-0.072083,-0.038818,-0.013184,-0.134766,0.104721,0.044922,-0.064453,-0.161621,0.134521,-0.044861,0.041077,-0.093018,-0.095062,0.066650,-0.001845,0.059570,0.055176,...,-0.039062,0.051270,-0.111328,-0.077881,0.323242,-0.087402,-0.247559,-0.042236,-0.142822,-0.224609,-0.012228,0.104004,0.133789,0.095703,0.018311,-0.012207,-0.246094,-0.410156,-0.019043,0.187866,0.017578,-0.022949,0.203613,-0.000488,-0.057617,-0.105225,-0.250488,-0.024414,-0.146973,-0.065430,-0.040649,-0.068970,-0.103516,0.040771,-0.079834,-0.018555,0.019287,-0.062561,0.144287,0.009277
4,0.121094,-0.046875,0.035889,0.283203,-0.166016,-0.080566,0.061768,-0.484375,-0.094727,-0.004883,-0.134766,0.113281,0.212891,0.099121,-0.134766,-0.225586,-0.242188,0.156250,-0.069824,-0.218750,-0.147461,0.298828,-0.010193,-0.221680,-0.225586,-0.031738,-0.173828,0.046875,0.025024,-0.221680,-0.043945,0.241211,0.125977,0.016235,-0.108398,0.068359,0.363281,-0.004944,-0.088379,0.222656,...,0.160156,0.039551,-0.096191,0.247070,0.066895,-0.038574,-0.061035,0.255859,0.105957,-0.002670,0.085938,0.082520,-0.107910,0.010986,0.408203,-0.026733,-0.117188,-0.230469,0.040527,0.165039,-0.032227,0.137695,0.281250,-0.050781,0.007111,0.011780,0.069824,-0.184570,-

In [24]:
#one hot encoding the target product categories
target_one_hot_encoded = pd.get_dummies(new_data['category'])

In [26]:
#merging the vectorized and one hot encoded dataframes.
dataset = pd.merge(vectorized_df,target_one_hot_encoded,left_index=True, right_index=True)

In [28]:
#random shuffling the data before splitting
dataset_shuffled = dataset.reindex(np.random.permutation(dataset.index))

In [29]:
dataset_shuffled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,accessories,apparel,budder,cartridge,concentrates,edibles,flower,gear,gummies,pods,pre-roll,rosin,shatter,topicals,vape-pens
6683,-0.011230,0.021159,-0.026560,0.080892,-0.102946,-0.023519,0.019165,-0.077962,0.104268,0.020630,0.030680,-0.114827,-0.044120,-0.094991,-0.154826,0.111796,-0.081116,0.108988,-0.004522,-0.150859,0.063702,-0.007568,-0.057617,0.064982,-0.122082,-0.030998,-0.093455,0.180786,0.152140,-0.125529,-0.161255,-0.016276,-0.099482,0.109375,0.000488,0.121419,-0.021847,0.065084,0.043050,0.023071,...,0.073873,-0.109456,-0.005127,-0.143005,0.060384,-0.100321,-0.047760,0.016032,0.076335,0.084391,-0.015544,0.093225,-0.000895,0.015747,-0.057007,-0.005229,0.089152,-0.020589,0.061157,-0.033773,-0.190511,0.125000,-0.103973,-0.077230,0.054138,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
10192,-0.117798,0.140808,0.128899,0.108286,-0.016968,0.103271,-0.041016,-0.076111,-0.023071,0.115234,-0.157959,-0.019043,-0.134173,-0.026123,-0.168274,0.003418,-0.251251,0.124481,-0.119141,-0.223503,-0.016602,-0.049866,-0.054077,0.025146,-0.122925,-0.056671,-0.096771,0.113342,-0.197334,-0.139069,-0.008057,-0.006531,-0.164185,0.160095,-0.305054,0.087555,0.066132,0.039948,-0.012878,0.145699,...,-0.246582,-0.069336,-0.013672,-0.074219,0.114685,-0.064697,0.047943,0.173561,-0.018768,-0.067383,-0.395508,0.146362,0.262329,-0.049408,0.309692,-0.067993,-0.013489,-0.060211,0.117643,0.034180,-0.070450,-0.013237,0.168518,-0.021606,0.097168,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9800,0.203613,0.157552,0.133341,0.082031,-0.123820,-0.094116,-0.100952,-0.170654,0.123698,0.069824,-0.141276,-0.177165,0.025879,-0.011393,-0.077962,0.101761,0.036499,0.091309,0.140788,-0.201009,-0.029460,-0.028300,0.023763,0.000814,-0.084310,-0.019694,-0.251953,0.261393,0.205566,-0.057943,0.053874,-0.029928,-0.055013,0.241211,0.033854,0.051758,-0.035807,0.066732,0.031067,0.041667,...,0.020182,-0.044474,-0.036621,-0.071960,0.212077,-0.149740,0.001302,-0.047526,-0.169596,0.095947,-0.024821,-0.198242,0.048055,-0.047892,0.062337,-0.085042,-0.082397,0.052653,0.026367,-0.024414,-0.274089,-0.167399,0.051676,-0.026123,0.060547,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8034,-0.127930,0.093262,-0.020874,-0.122559,0.250000,0.071289,0.037842,-0.089355,0.051270,0.197266,0.125000,-0.014526,0.041748,0.113770,-0.013123,0.056641,-0.030273,0.073242,0.124023,0.110840,0.177734,0.024902,0.056152,-0.084473,-0.090820,0.056396,-0.037598,0.149414,-0.003937,-0.107422,-0.161133,-0.038086,-0.003525,0.015320,0.004089,0.057373,0.095703,-0.116699,0.180664,0.000786,...,0.013489,-0.093750,-0.166016,0.013184,-0.105957,-0.044922,-0.337891,0.130859,0.016724,0.080566,-0.109375,-0.077148,-0.139648,-0.141602,-0.010376,-0.009888,-0.117676,0.041748,-0.142578,-0.129883,-0.100098,-0.091797,0.022339,-0.081543,-0.155273,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8626,-0.369141,0.040771,0.100098,0.179443,0.091797,-0.078857,0.221191,-0.021851,-0.217285,0.253418,-0.382812,0.027222,-0.158691,0.001221,-0.097168,-0.012207,-0.273804,0.045288,-0.009766,-0.278320,-0.044678,-0.043915,0.002930,-0.017578,-0.039764,-0.391602,-0.111328,0.314941,-0.064224,-0.118927,-0.023926,0.162598,-0.212646,0.086060,-0.383301,0.018066,0.061035,-0.016968,0.011993,0.075089,...,-0.150391,-0.244385,0.145752,0.018066,0.035645,-0.170532,0.126694,0.361328,0.086487,-0.334961,-0.243896,0.041443,0.235352,-0.159668,0.021851,-0.078003,-0.111816,0.079590,0.118896,0.136963,0.035126,-0.240845,0.031982,0.234619,0.035156,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10581,-0.104980,0.051107,0.142741,0

In [30]:
target_one_hot_encoded.columns

Index(['accessories', 'apparel', 'budder', 'cartridge', 'concentrates',
       'edibles', 'flower', 'gear', 'gummies', 'pods', 'pre-roll', 'rosin',
       'shatter', 'topicals', 'vape-pens'],
      dtype='object')

In [31]:
#splitting of data
from sklearn.model_selection import train_test_split

target = target_one_hot_encoded.columns
X = dataset_shuffled.loc[:,~dataset_shuffled.columns.isin(target)]
y = dataset_shuffled.loc[:,dataset_shuffled.columns.isin(target)]

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.75,test_size=0.25,random_state=101)

In [ ]:
#Data Modeling